In [25]:
import requests
from sklearn.cluster import KMeans
import pandas as pd

char_codes = {}
inc = 0


def code(text: str):
    global inc
    c = char_codes.get(text, None)
    if c is None:
        char_codes[text] = inc
        inc += 1
        return char_codes[text]
    return c


def convert(data: list[dict]):
    l = []
    for item in data:
        for k, v in item.items():
            l.append([code(k), code(v)])
    return l

def load_products():
    is_exit = False
    data = {
        "id": [],
        "name": [],
        "price": [],
        "tags": [],
        # "characteristics": [],
        "is_service": [],
    }
    i = 0
    while not is_exit:
        res = requests.get(
            f"http://46.243.226.43:6100/stores/products/all?offset={i}"
        ).json()
        if res.get("count", None) is None:
            break
        for item in res["items"]:
            data["id"].append(item["id"])
            data["name"].append(item["name"])
            data["is_service"].append(1 if item["is_service"] else 0)
            data["price"].append(
                item["price_info"]["price_min"] + item["price_info"]["price_max"]
            )
            data["tags"].append([tag["tag_id"] for tag in item["product_tag_links"]])
            # data["characteristics"].append(convert(item["details"]["characteristics"]))
        i += 1
    max_price = max(data["price"])
    data["price"] = [price / max_price for price in data["price"]]
    return data


data_db= load_products()
print(data_db)

{'id': [64, 16, 14, 20, 48, 59, 10, 8, 52, 66, 31, 23, 5, 39, 56, 33, 9, 63, 29, 57, 50, 62, 53, 21, 65, 68, 15, 58, 30, 22, 60, 67, 36, 32, 19, 24, 26, 44, 28, 46, 61, 17, 25, 42, 43, 54, 37, 27, 49, 45, 34, 35, 51, 40, 18, 38, 55, 41, 47, 11], 'name': ['SSD-накопитель Samsung 970 EVO Plus', 'TechnoGrand Smart Watch x7 pro', 'Xiaomi Redmi Note 12S 8', 'Английский язык. Контрольные задания. 7 класс. ФГОС / Английский в фокусе. Spotlight / Просвещение', 'Букет из красных роз', 'Видеокарта NVIDIA GeForce RTX 3080', 'Гантели Reebok Rubber Hex Dumbbells', "Горный Велосипед Scott Scale 970 29'' 2023 dark grey (L), 29, 2023", "Картофель 'Ред Скарлет'", 'Клавиатура Logitech G915 TKL', 'Комплект стульев _SOKOLTEC_HW900_, 4 шт. Белый.', 'Крестик Серебро 925 Золочение Распятие Молитва Да воскреснет Бог ', 'Кроссовки Nike Air Max 270', "Круиз 'Карибское море' (эконом)", "Курица 'Домашняя'", "Куртка 'Спортивный стиль'. Синий S", 'Лыжи Fischer RCS Carbonlite Skate Plus', 'Маршрутизатор ASUS RT-AX88

In [26]:
# Загрузка данных о товарах из csv файла
data = pd.DataFrame.from_dict(data_db)

# Выбор признаков для кластеризации (например, цена, размер, материал)
X = data[["is_service", "tags", "price"]]
#  "characteristics", 
# Определение количества кластеров
kmeans = KMeans(n_clusters=3)

# Обучение модели на выбранных признаках
kmeans.fit(X)

# Прогнозирование принадлежности товаров к кластерам
labels = kmeans.predict(X)

# Добавление меток кластеров в исходный датасет
data["cluster"] = labels

# Сохранение результатов в новый csv файл
print(data.to_dict())

ValueError: setting an array element with a sequence.